## *Setup required for Playground*

The online playground requires a little bit of code to run first. 

In [ ]:
import piplite
await piplite.install('kloppy')
await piplite.install('pandas')

# Event data

One of the main benefits of working with kloppy is that it loads metadata with the event data. This metadata includes teams (name, ground and provider id) and players (name, jersey number, optional position and provider id). Using this metadata, it becomes very easy to create an analysis that is usable by humans, because it includes names instead of only numbers.

This section shows how metadata is organized and some use-cases.

## Loading statsbomb data

The datasets module of kloppy makes it trivial to load statsbomb data. Keep in mind that by using the data you accept the license of the open-data project.


In [ ]:
from kloppy import statsbomb

dataset = statsbomb.load_open_data(event_types=["pass", "shot"])

## Exploring metadata

kloppy always loads the metadata for you and makes it available at the `metadata` property. 

In [ ]:
metadata = dataset.metadata
home_team, away_team = metadata.teams

After loading the data, the metadata can be used to iterate over teams and players. By default `metadata.teams` contain `[HomeTeam, AwayTeam]`. `Team` and `Player` entities have the `__str__` magic method implemented to help you cast it to a string. When you want to 

In [ ]:
print(f"{home_team.ground} - {home_team}")
print(f"{away_team.ground} - {away_team}")

In [ ]:
[f"{player} ({player.jersey_no})" for player in home_team.players]


In [ ]:
# get provider id for team
f"statsbomb team id: {home_team.team_id} - {away_team.team_id}"

In [ ]:
# same for the players
[f"{player} id={player.player_id}" for player in metadata.teams[0].players]


In [ ]:
# get player from first event
player = dataset.events[0].player
print(player)
print(player.team)
print(f"Teams are comparable? {player.team == away_team}")

The `Team` and `Player` entities also contain the magic methods to use those keys in dictionaries or use them in sets. This makes it easy to do some calculations, and show the results without mapping the player_id to a name.

In [ ]:
from collections import defaultdict

passes_per_player = defaultdict(list)
for event in dataset.events:
    if event.event_name == "pass":
        passes_per_player[event.player].append(event)
        
for player, passes in passes_per_player.items():
    print(f"{player} has {len(passes)} passes")

Now let's filter on home_team.

In [ ]:
for player, passes in passes_per_player.items():
    if player.team == home_team:
        print(f"{player} has {len(passes)} passes")

## Use metadata when transforming to pandas dataframe

The metadata can also be used when transforming a dataset to a pandas dataframe. The `additional_columns` argument should be passed to `to_pandas`. 

In [ ]:

dataframe = dataset.to_pandas(
    additional_columns={
        'player_name': lambda event: str(event.player),
        'team_name': lambda event: str(event.player.team)
    }
)

dataframe[[
    'event_id', 'event_type', 'result', 'timestamp', 'player_id', 
    'player_name', 'team_name'
]].head()

## Attribute transformers

Attribute transformer make it possible to add predefined attributes to a dataset. The attributes are calculated during export to a pandas DataFrame. Kloppy does provide some Transformers like one to calculate the angle to the goal, and one to calculate the distance to the goal. When you need additional Transformers you can write your one by providing a `Callable` to `to_df`. 

In [ ]:
from kloppy import statsbomb

from kloppy.domain.services.transformers.attribute import (
    BodyPartTransformer, AngleToGoalTransformer, DistanceToGoalTransformer
)

dataset = statsbomb.load_open_data(
    event_types=["pass", "shot"], 
    coordinates="statsbomb"
)

dataset.to_df(
    AngleToGoalTransformer(),
    DistanceToGoalTransformer()
)

In [ ]:
event = dataset.events[0]

transformer = BodyPartTransformer(encoding="one-hot")
print(transformer(event))


transformer = AngleToGoalTransformer()
transformer(event)

## Wildcard

When you want to export a set of attributes you can specify a wildcard pattern. This pattern is matched against all default (exported by the `Default` Transformer) attributes.

In [ ]:
dataset.to_df(
    'period_id',
    'timestamp',
    '*coordinates*',
)

## User-defined Transformers

Transformers are nothing more than a function which accepts a `Event` and returns `Dict` (`Callable[[Event], Dict])`). The Transformers provided by kloppy are actually classes that define a `__call__` method. You can also use a `lambda` function or any other function to transform attributes.

When you use named attributes (specified using a keyword argument) the returned value can be any type (`Callable[[Event], Any]`). 

In [ ]:
import random

dataset.to_df(
    # Unnamed transformer must always be defined as a Callable. The function must return a Dictionary
    lambda event: {'period': event.period.id, 'timestamp': event.timestamp},
    
    # Named transformer can be specified as a constant
    some_columns=1234,
    
    # Or as a callable
    other_column=lambda x: random.randint(0, 255)
)

## to_records

Under the hood the `to_df` method uses the `to_records` method. 

In [ ]:
records = dataset.to_records(
    # Unnamed transformer must always be defined as a Callable. The function must return a Dictionary
    lambda event: {'period': event.period.id, 'timestamp': event.timestamp},
    
    # Named transformer can be specified as a constant
    some_columns=1234,
    
    # Or as a callable
    other_column=lambda x: random.randint(0, 255)
)
records[:10]